In [15]:
import pandas as pd

### Load Data

In [26]:
years = ['2018','2019','2020','2021']
demands = [pd.read_excel(open('demands.xls', 'rb'), sheet_name=year) for year in years]

### Pre Processing

In [27]:
for i in range(len(years)):
     demands[i]["YEAR"]=int(years[i])
total_demands = pd.concat(demands)

In [28]:
total_demands

,SKU_ID,'01','02','03','04','05','06','07','08','09','10','11','12',TOTAL_FOR_YEAR,AVG_PER_MONTH,TRIGGER_QTY,MAXIMUM_QTY,QTY_ON_HAND,YEAR
0,NU802536,2839.0,1513.0,1639.0,1466.0,1484.0,1345.0,1812.0,1271.0,1625.0,1652.0,1297.0,1285.0,19228,1602.3,150.0,432,408,2018
1,14571184,12141.0,11562.0,10361.0,13413.0,9324.0,12707.0,8893.0,8776.0,7812.0,15865.0,17097.0,17987.0,145938,12161.5,1152.0,2304,1104,2018
2,RB038125,400.0,206.0,493.0,131.0,308.0,242.0,126.0,NaN,92.0,205.0,180.0,168.0,2551,212.6,24.0,48,0,2018
3,DS088381,954.0,636.0,678.0,330.0,528.0,360.0,546.0,492.0,534.0,948.0,3504.0,1026.0,10536,878.0,144.0,240,126,2018
4,DS088459,49.0,22.0,83.0,26.0,29.0,27.0,40.0,20.0,29.0,40.0,29.0,99.0,493,41.1,60.0,120,127,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3007,GS703250,NaN,NaN,NaN,NaN,NaN,NaN,12.0,99.0,21.0,39.0,24.0,23.0,218,18.2,56.0,112,60,2021
3008,WT420409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3,0.3,20.0,40,47,2021
3009,SQ336060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,44,3.7,28.0,56,30,2021
3010,0SK00441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,17,1.4,240.0,480,259,2021


In [29]:
assuta_items = pd.read_excel("Assuta_items.xlsx")
assuta_items

,SKU_ID,Is_Assuta
0,AS100016,Y
1,AS120257,Y
2,AS110041,Y
3,AS180293,Y
4,AS040107,Y
...,...,...
1285,AS400185,Y
1286,AS010122,Y
1287,AS000201,Y
1288,ASA00006,Y


In [30]:
total_demands = total_demands[~total_demands["SKU_ID"].isin(assuta_items["SKU_ID"])].copy()

In [31]:
total_demands.fillna(0,inplace=True)

,SKU_ID,'01','02','03','04','05','06','07','08','09','10','11','12',TOTAL_FOR_YEAR,AVG_PER_MONTH,TRIGGER_QTY,MAXIMUM_QTY,QTY_ON_HAND,YEAR
0,NU802536,2839.0,1513.0,1639.0,1466.0,1484.0,1345.0,1812.0,1271.0,1625.0,1652.0,1297.0,1285.0,19228,1602.3,150.0,432,408,2018
1,14571184,12141.0,11562.0,10361.0,13413.0,9324.0,12707.0,8893.0,8776.0,7812.0,15865.0,17097.0,17987.0,145938,12161.5,1152.0,2304,1104,2018
2,RB038125,400.0,206.0,493.0,131.0,308.0,242.0,126.0,0.0,92.0,205.0,180.0,168.0,2551,212.6,24.0,48,0,2018
3,DS088381,954.0,636.0,678.0,330.0,528.0,360.0,546.0,492.0,534.0,948.0,3504.0,1026.0,10536,878.0,144.0,240,126,2018
4,DS088459,49.0,22.0,83.0,26.0,29.0,27.0,40.0,20.0,29.0,40.0,29.0,99.0,493,41.1,60.0,120,127,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3007,GS703250,0.0,0.0,0.0,0.0,0.0,0.0,12.0,99.0,21.0,39.0,24.0,23.0,218,18.2,56.0,112,60,2021
3008,WT420409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3,0.3,20.0,40,47,2021
3009,SQ336060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,44,3.7,28.0,56,30,2021
3010,0SK00441,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,17,1.4,240.0,480,259,2021


In [32]:
total_demands

,SKU_ID,'01','02','03','04','05','06','07','08','09','10','11','12',TOTAL_FOR_YEAR,AVG_PER_MONTH,TRIGGER_QTY,MAXIMUM_QTY,QTY_ON_HAND,YEAR
0,NU802536,2839.0,1513.0,1639.0,1466.0,1484.0,1345.0,1812.0,1271.0,1625.0,1652.0,1297.0,1285.0,19228,1602.3,150.0,432,408,2018
1,14571184,12141.0,11562.0,10361.0,13413.0,9324.0,12707.0,8893.0,8776.0,7812.0,15865.0,17097.0,17987.0,145938,12161.5,1152.0,2304,1104,2018
2,RB038125,400.0,206.0,493.0,131.0,308.0,242.0,126.0,NaN,92.0,205.0,180.0,168.0,2551,212.6,24.0,48,0,2018
3,DS088381,954.0,636.0,678.0,330.0,528.0,360.0,546.0,492.0,534.0,948.0,3504.0,1026.0,10536,878.0,144.0,240,126,2018
4,DS088459,49.0,22.0,83.0,26.0,29.0,27.0,40.0,20.0,29.0,40.0,29.0,99.0,493,41.1,60.0,120,127,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3007,GS703250,NaN,NaN,NaN,NaN,NaN,NaN,12.0,99.0,21.0,39.0,24.0,23.0,218,18.2,56.0,112,60,2021
3008,WT420409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3,0.3,20.0,40,47,2021
3009,SQ336060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,44,3.7,28.0,56,30,2021
3010,0SK00441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,17,1.4,240.0,480,259,2021


In [33]:
# Add quarters columns
total_demands["Q1"] = total_demands["'01'"] + total_demands["'02'"] + total_demands["'03'"]
total_demands["Q2"] = total_demands["'04'"] + total_demands["'05'"] + total_demands["'06'"]
total_demands["Q3"] = total_demands["'07'"] + total_demands["'08'"] + total_demands["'09'"]
total_demands["Q4"] = total_demands["'10'"] + total_demands["'11'"] + total_demands["'12'"]

In [25]:
total_demands

,SKU_ID,'01','02','03','04','05','06','07','08','09',...,TOTAL_FOR_YEAR,AVG_PER_MONTH,TRIGGER_QTY,MAXIMUM_QTY,QTY_ON_HAND,YEAR,Q1,Q2,Q3,Q4
0,NU802536,2839.0,1513.0,1639.0,1466.0,1484.0,1345.0,1812.0,1271.0,1625.0,...,19228,1602.3,150.0,432,408,2018,5991.0,4295.0,4708.0,4234.0
1,14571184,12141.0,11562.0,10361.0,13413.0,9324.0,12707.0,8893.0,8776.0,7812.0,...,145938,12161.5,1152.0,2304,1104,2018,34064.0,35444.0,25481.0,50949.0
2,RB038125,400.0,206.0,493.0,131.0,308.0,242.0,126.0,NaN,92.0,...,2551,212.6,24.0,48,0,2018,1099.0,681.0,NaN,553.0
3,DS088381,954.0,636.0,678.0,330.0,528.0,360.0,546.0,492.0,534.0,...,10536,878.0,144.0,240,126,2018,2268.0,1218.0,1572.0,5478.0
4,DS088459,49.0,22.0,83.0,26.0,29.0,27.0,40.0,20.0,29.0,...,493,41.1,60.0,120,127,2018,154.0,82.0,89.0,168.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3007,GS703250,NaN,NaN,NaN,NaN,NaN,NaN,12.0,99.0,21.0,...,218,18.2,56.0,112,60,2021,NaN,NaN,132.0,86.0
3008,WT420409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,0.3,20.0,40,47,2021,NaN,NaN,NaN,NaN
3009,SQ336060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,44,3.7,28.0,56,30,2021,NaN,NaN,NaN,NaN
3010,0SK00441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17,1.4,240.0,480,259,2021,NaN,NaN,NaN,NaN


### Winters Model

In [ ]:
# %pip install statsmodels
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
# Set the value of Alpha and define m (Time Period)
m = 12
alpha = 1/(2*m)

In [ ]:
airline[‘HWES3_MUL’] = ExponentialSmoothing(airline[‘Thousands of Passengers’],trend=’mul’,seasonal=’mul’,
                                            seasonal_periods=12).fit().
fittedvalues
airline[[‘Thousands of Passengers’,’HWES3_ADD’,’HWES3_MUL’]].plot(title=’Holt Winters Triple Exponential Smoothing: Additive and Multiplicative Seasonality’);

In [ ]:
# a=list(total_demands["SKU_ID"])
# for i in a:
#     if "AS" in str(i):
#         print(i)

In [ ]:
# Group A - find the ~20% with highest demands
# for every product, calculate the average demand for the 4 years
annual_avg = ((total_demands.groupby("SKU_ID")["TOTAL_FOR_YEAR"].sum())/4).to_frame()
annual_avg.columns = ['YEARLY_AVG']
annual_avg.reset_index(inplace=True)
# annual_avg.head()

In [ ]:
annual_avg